In [ ]:
!pip freeze | grep scikit-learn

In [1]:
import pickle
import pandas as pd

In [2]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [3]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [4]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

In [5]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [6]:
duration_std = round(y_pred.std(), 2)
print(f"Standard deviation of trip duration in February 2022: {duration_std}")

Standard deviation of trip duration in February 2022: 5.28


In [7]:
from pathlib import Path

year = 2022
month = 2
taxi_type = 'yellow'

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')


df_result = df.copy(deep=True)[['ride_id']]
df_result['y_pred'] = y_pred


output_folder = Path(f'output/{taxi_type}')
output_folder.mkdir(parents=True, exist_ok=True)
output_file = Path(f'{year:04d}-{month:02d}.parquet')
output_full = output_folder / output_file

df_result.to_parquet(
    output_full,
    engine='pyarrow',
    compression=None,
    index=False
)